<a href="https://colab.research.google.com/github/AliBenovaa/IANNwTF_Group24/blob/main/Homewrok10Group24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "tensorflow-text==2.8.*"

In [2]:
# imports
import os
import tensorflow as tf
import re
import numpy as np
import tensorflow_text as tf_text
from collections import Counter
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


# define the hyperparameters
vocab_size = 10000
embedding_size = 64

In [3]:
# bash code to mount the drive
import os
from google.colab import drive
drive.mount ("/content/drive")
os.chdir ("drive/MyDrive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Load text from file
with open('bible.txt', 'r') as file:
    text = file.read()


tokens = text.lower().replace('\n', ' ').replace('[^\w\s]','').split()
print(len(tokens))

821122


In [5]:
from collections import Counter

# Load text from file
with open('bible.txt', 'r') as file:
    text = file.read()


text = text.lower().replace('\n', ' ').replace('[^\w\s]','')

# Split text into words
tokens =tf_text.WhitespaceTokenizer().split(text)

#save tokens as list
tokens = list(tokens.numpy())
#convert the tokens from uft8 to string
tokens  = [x.decode('utf8') for x in tokens]

# Count frequency of each word
word_counts = Counter(tokens)

# Keep only the 1000 most common words
most_common_words = [word for word, count in word_counts.most_common(1000)]

# Assign each word an integer value
word_index = {word: i for i, word in enumerate(most_common_words)}

# Print word-to-integer mapping
print(word_index)


{'the': 0, 'and': 1, 'of': 2, 'to': 3, 'that': 4, 'in': 5, 'he': 6, 'shall': 7, 'unto': 8, 'for': 9, 'i': 10, 'his': 11, 'a': 12, 'they': 13, 'be': 14, 'is': 15, 'with': 16, 'not': 17, 'all': 18, 'thou': 19, 'lord': 20, 'it': 21, 'thy': 22, 'was': 23, 'which': 24, 'my': 25, 'but': 26, 'their': 27, 'have': 28, 'ye': 29, 'will': 30, 'from': 31, 'as': 32, 'them': 33, 'him': 34, 'are': 35, 'when': 36, 'upon': 37, 'were': 38, 'this': 39, 'by': 40, 'out': 41, 'god': 42, 'said': 43, 'hath': 44, 'then': 45, 'there': 46, 'him,': 47, 'into': 48, 'had': 49, 'came': 50, 'man': 51, 'on': 52, 'up': 53, 'son': 54, 'one': 55, 'we': 56, 'king': 57, 'me': 58, 'your': 59, 'before': 60, 'lord,': 61, 'an': 62, 'said,': 63, 'against': 64, 'shalt': 65, 'them,': 66, 'come': 67, 'if': 68, 'children': 69, 'at': 70, 'so': 71, 'her': 72, 'thee': 73, 'also': 74, 'let': 75, 'no': 76, 'even': 77, 'made': 78, 'saying,': 79, 'went': 80, 'house': 81, 'go': 82, 'every': 83, 'people': 84, 'do': 85, 'thee,': 86, 'because'

In [6]:
#Step 2: create vocabulary
char_to_int = dict((c,i) for i,c in enumerate(most_common_words))
int_to_char = dict((i,c) for i,c in enumerate(most_common_words))
print("Vocabulary:",int_to_char)

Vocabulary: {0: 'the', 1: 'and', 2: 'of', 3: 'to', 4: 'that', 5: 'in', 6: 'he', 7: 'shall', 8: 'unto', 9: 'for', 10: 'i', 11: 'his', 12: 'a', 13: 'they', 14: 'be', 15: 'is', 16: 'with', 17: 'not', 18: 'all', 19: 'thou', 20: 'lord', 21: 'it', 22: 'thy', 23: 'was', 24: 'which', 25: 'my', 26: 'but', 27: 'their', 28: 'have', 29: 'ye', 30: 'will', 31: 'from', 32: 'as', 33: 'them', 34: 'him', 35: 'are', 36: 'when', 37: 'upon', 38: 'were', 39: 'this', 40: 'by', 41: 'out', 42: 'god', 43: 'said', 44: 'hath', 45: 'then', 46: 'there', 47: 'him,', 48: 'into', 49: 'had', 50: 'came', 51: 'man', 52: 'on', 53: 'up', 54: 'son', 55: 'one', 56: 'we', 57: 'king', 58: 'me', 59: 'your', 60: 'before', 61: 'lord,', 62: 'an', 63: 'said,', 64: 'against', 65: 'shalt', 66: 'them,', 67: 'come', 68: 'if', 69: 'children', 70: 'at', 71: 'so', 72: 'her', 73: 'thee', 74: 'also', 75: 'let', 76: 'no', 77: 'even', 78: 'made', 79: 'saying,', 80: 'went', 81: 'house', 82: 'go', 83: 'every', 84: 'people', 85: 'do', 86: 'thee,

In [7]:
freq = [word_counts.get(key) for key in list(int_to_char.values())]

In [8]:
#remove uncommon token from bible text
def keep_tokens(tokens, tokens_to_keep):
    tokens_to_keep_set = set(tokens_to_keep)
    return [token for token in tokens if token in tokens_to_keep_set]

new_text = keep_tokens(tokens,most_common_words)

#translate bible into our vocabulary                  
trans_text = np.vectorize(lambda x: char_to_int.get(x, 0))(new_text)


In [9]:
def create_pairs(tokens, word_index, window_size=4):
    pairs = []
    for i, word in enumerate(tokens):
        target_indices = [j for j in range(max(1, i - window_size), min(len(tokens), i + window_size + 1)) if j != i][:window_size]
        for j in target_indices:
            input_word = word_index.get(tokens[i], 1)
            target_word = word_index.get(tokens[j], 1)
            pairs.append((input_word, target_word))
    return pairs

In [10]:
# Create input-target pairs
pairs = create_pairs(tokens, word_index)

In [11]:
# Print the first 10 pairs
print(pairs[:10])

[(0, 211), (0, 442), (0, 2), (0, 1), (211, 442), (211, 2), (211, 1), (211, 153), (442, 211), (442, 2)]


In [12]:
# convert the pairs into a tensor of shape (num_pairs, 2)
#pairs = tf.convert_to_tensor(pairs, dtype=tf.int64)


# create a dataset from the pairs tensor
train_ds = tf.data.Dataset.from_tensor_slices(pairs)

# shuffle the batches
train_ds = train_ds.shuffle(1000)

# batch the dataset
batch_size = 128
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(20).cache()

In [13]:
class SkipGram(tf.keras.Model):
  """
  Encoder class
  Defines a model using LSTM layers and read_in and output layers to make a final prediction, inheriting from tf.keras.Model.
  """

  def __init__(self, number_of_words, embedding_size):
    """ 
    Initializes the model.
    In the transition layer we reduce the number of channels to half of the existing channels.
    Parameters
    ----------
    latent_dim : int
         size of the embedding
    """

    super(SkipGram, self).__init__()
    self.number_of_words = number_of_words
    self.embedding_size = embedding_size

  def build(self, input_shape):
    self.embedding_matrix = self.add_weight(shape=(self.number_of_words, self.embedding_size))
    self.scores_matrix = self.add_weight(shape=(self.number_of_words, self.embedding_size))
    self.scores_bias = self.add_weight(shape=(self.number_of_words,), initializer="zeros")

  #@tf.function
  def call(self, inputs, labels = None):
    """ 
    Computes a forward step with the given data
    Parameters
    ----------
    inputs : tf.Tensor
        the input for the model
   
    Returns
    -------
    x : tf.Tensor
        the output of the model which is only the last time step
    """
    embeddings = tf.nn.embedding_lookup(self.embedding_matrix, inputs)
    labels = tf.expand_dims(labels, axis=-1)
    
    labels = tf.cast(labels, dtype=tf.int64)
    sampler = tf.nn.fixed_unigram_candidate_sampler(
        labels,
        num_true=1,
        num_sampled=256,
        unique=True,
        range_max=self.number_of_words,
        unigrams=freq)

    loss = tf.nn.nce_loss(
        weights = self.scores_matrix,
        biases = self.scores_bias,
        labels = labels,
        inputs = embeddings,
        num_sampled = 256,
        num_classes = self.number_of_words,
        sampled_values = sampler
    )

    loss = tf.reduce_mean(loss)
    return loss

In [14]:
#calculate and pring the nearest neighbours according to the cosine similarity of each epoch
def print_nearest_neighbours(tracked_words, k, embeddings):
  
  scaled_embeddings = embeddings / np.sqrt(np.sum(embeddings**2, axis=1, keepdims=True))
  for i in range(len(tracked_words)):
    word_index = char_to_int[tracked_words[i]]
    embedding = scaled_embeddings[word_index, :]
    cosine_similarity = np.matmul(scaled_embeddings, embedding)
    k_nearest_neighbours = np.argsort(-cosine_similarity)[:k]
    print("Nearest neighbours of " + tf.compat.as_str_any(tracked_words[i]) + " are: ")
   
    for i in k_nearest_neighbours:
      print(tf.compat.as_str_any(int_to_char[i]))

In [15]:
#@tf.function
def train_step(model, input, target, optimizer):
  """ Computes a train step with the given data
  Parameters
  ----------
  model : respective model class (super: tf.keras.Model)
    the model to perform train step
  inputs : tf.Tensor
    the input for the model
  target : tf.Tensor
    the target label
  loss_function : tf.keras.losses
    given loss function of respective tensorflow classes
  optimizer : tf.keras.optimizers
    given optimizer of respective tensorflow classes
  Returns
  -------
  loss : tf.Tensor
      the current loss of the model
  """
  with tf.GradientTape() as tape:
     # Ensure that target is a tensor of shape (batch_size, 1)
    target = tf.expand_dims(target, axis=-1)
    loss = model(input, target)
    #loss = model(input)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

In [ ]:
tf.keras.backend.clear_session()

# Definition of the hyperparameters
num_epochs = 5
learning_rate = tf.constant(0.01, dtype=tf.float32)

# Tracked words and number of neighbours that will be shown
tracked_words = [b'holy', b'father', b'wine', b'poison', b'love', b'strong', b'day']
k = 10

number_of_words = vocab_size
# Initialize the model
model = SkipGram(number_of_words, embedding_size)

# Initialize the optimizer:
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize numpy arrays for later visualization
train_losses = np.empty(0)

# check how model performs on train data once before we begin
#train_loss = test(model, train_ds, global_loss_function)
#train_losses = np.append(train_losses, train_loss)
epoch_loss_agg = np.empty(0)
# We train for num_epochs epochs.
for epoch in range(num_epochs):
    # Display accuracy at the beginning of each epoch
    #print(f'Epoch: {str(epoch)} starting with test loss {test_losses[-1]}')

    for example in train_ds:
      input = example[:-1]
      target = example[-1]
      input = tf.reshape(input, [-1])
      train_loss = train_step(model, input, target, optimizer)
      epoch_loss_agg = np.append(epoch_loss_agg, train_loss)
        

    # Track training loss
    train_losses = np.append(train_losses, tf.reduce_mean(train_loss))
    
    # Display train loss
    print(f'Epoch: {str(epoch)} finishing with train loss {train_losses[-1]}')
    print(" ")

    # Print nearest neighbours
    embeddings = model.trainable_variables[0].numpy()
    print_nearest_neighbours(tracked_words, k, embeddings)
    print(" ")

# Visualize accuracy and loss for training and test data
plt.figure()
line1, = plt.plot(train_losses)
plt.xlabel("Training steps")
plt.ylabel("Loss")
plt.legend((line1), ("training loss"))
plt.show()